In [1]:
# step1_parse_and_load.py
from pathlib import Path
import collections

def parse_line(line):
    parts = line.strip().split()
    if len(parts) < 2:
        return None
    email_id = parts[0]
    label = parts[1].lower()
    token_counts = {}
    rest = parts[2:]
    i = 0
    while i < len(rest):
        token = rest[i]
        if i + 1 < len(rest):
            try:
                cnt = int(rest[i+1])
                i += 2
            except ValueError:
                cnt = 1
                i += 1
        else:
            cnt = 1
            i += 1
        token_counts[token] = token_counts.get(token, 0) + cnt
    return email_id, label, token_counts

def load_dataset(path):
    emails = []
    with open(path, 'r', encoding='utf8', errors='ignore') as f:
        for line in f:
            parsed = parse_line(line)
            if parsed:
                emails.append(parsed)
    return emails

if __name__ == "__main__":
    train = load_dataset("/content/train")
    test  = load_dataset("/content/test")
    print("Train examples:", len(train))
    print("Test examples: ", len(test))
    # print first parsed example for quick check
    if train:
        print("First train example:", train[0])

Train examples: 5490
Test examples:  1000
First train example: ('/000/003', 'ham', {'need': 1, 'fw': 1, '35': 2, '39': 1, 'thanks': 1, 'thread': 2, '40': 1, 'copy': 1, 'else': 1, 'correlator': 1, 'under': 1, 'companies': 1, '25': 1, 'he': 2, '26': 2, '168': 1, '29': 2, 'content': 4, '1': 1, '6': 1, '5': 1, '4': 1, 'review': 2, 'we': 1, 'john': 3, '17': 1, 'use': 1, '15': 1, '20': 1, 'classes': 1, 'may': 1, 'a': 1, 'back': 1, 'l': 1, '01': 1, 'produced': 1, 'i': 1, 'yes': 1, '10': 2, '713': 2, 'v6': 1, 'p': 1, 'original': 2, 'doc': 2, '2001': 4, 'comments': 1, 'x': 3, 'week': 1, 'to': 6, 'harry': 2, '110': 1, 'smtpsvc': 1, 'has': 1, 'who': 1, 'sender': 1, 'would': 1, 'any': 2, 'jan': 5, 'be': 1, 'index': 1, 'text': 1, 'intended': 1, 'and': 2, 'urn': 1, 'email': 2, 'cc': 1, '1600': 1, 'can': 1, 're': 2, 'corp': 3, 'fri': 2, 'response': 1, '65': 1, 'plain': 1, 'confidential': 1, '853': 1, 'you': 6, 'mailto': 1, 'anything': 1, 'am': 1, 'our': 3, 'they': 1, 'for': 6, 'info': 1, 'of': 2, 'ar

In [2]:
import pandas as pd
columns = ['email_id', 'label', 'token_counts']
df_train = pd.DataFrame(train, columns=columns)
df_test = pd.DataFrame(test, columns=columns)


In [3]:
df_train.head()

,email_id,label,token_counts
0,/000/003,ham,"{'need': 1, 'fw': 1, '35': 2, '39': 1, 'thanks..."
1,/000/031,ham,"{'don': 1, 'kim': 5, 'dave': 1, '39': 1, 'cust..."
2,/000/044,ham,"{'to': 1, 'mimeole': 1, 'for': 1, 'kim': 3, '1..."
3,/000/057,ham,"{'to': 2, 'mimeole': 1, '110': 1, 'smtpsvc': 1..."
4,/000/069,ham,"{'smtp': 1, 'for': 3, 'visit': 1, 'its': 1, '0..."


# 2) Compute priors P(spam) and P(ham)

In [4]:
import collections

labels = [e[1] for e in train]
counter = collections.Counter(labels)
total = len(labels)
P_spam = counter.get('spam', 0) / total if total else 0.0
P_ham  = counter.get('ham',  0) / total if total else 0.0

print(f"Total train emails: {total}")
print(f"P(spam) = {P_spam:.6f}")
print(f"P(ham)  = {P_ham:.6f}")

Total train emails: 5490
P(spam) = 0.480874
P(ham)  = 0.519126


# 3) Build vocabulary and class word counts

In [5]:
unique_words = set()

for tokens in df_train["token_counts"]:
    unique_words.update(tokens.keys())

m = len(unique_words)# vocab size
print(f"Vocabulary size: {m}")

Vocabulary size: 997


In [6]:
df_ham = df_train[df_train["label"] == "ham"].reset_index(drop=True)

df_spam = df_train[df_train["label"] == "spam"].reset_index(drop=True)

print("ham count:", len(df_ham))
print("spam count:", len(df_spam))

ham count: 2850
spam count: 2640


In [7]:
total_words_spam = 0
total_words_ham = 0

for tokens in df_spam["token_counts"]:
    total_words_spam += sum(tokens.values())
print("word count spam:", total_words_spam)

for tokens in df_ham["token_counts"]:
    total_words_ham += sum(tokens.values())
print("word count ham:", total_words_ham)

word count spam: 876163
word count ham: 1254236


In [8]:
overall_df = pd.DataFrame(columns=['word', 'spam_count', 'ham_count' , 'p(word|spam)' , 'p(word|ham)'])
overall_df["word"] = list(unique_words)

In [9]:
overall_df.head()

,word,spam_count,ham_count,p(word|spam),p(word|ham)
0,forward,NaN,NaN,NaN,NaN
1,support,NaN,NaN,NaN,NaN
2,kitchen,NaN,NaN,NaN,NaN
3,heard,NaN,NaN,NaN,NaN
4,december,NaN,NaN,NaN,NaN


In [10]:
for i, row in overall_df.iterrows():
    word = row["word"]
    overall_df.at[i, "spam_count"] = sum(tokens.get(word, 0) for tokens in df_spam["token_counts"])

for i, row in overall_df.iterrows():
    word = row["word"]
    overall_df.at[i, "ham_count"] = sum(tokens.get(word, 0) for tokens in df_ham["token_counts"])

In [11]:
overall_df.head()

,word,spam_count,ham_count,p(word|spam),p(word|ham)
0,forward,1258,467,NaN,NaN
1,support,148,365,NaN,NaN
2,kitchen,4,522,NaN,NaN
3,heard,32,346,NaN,NaN
4,december,209,469,NaN,NaN


In [12]:
for i, row in overall_df.iterrows():
    word = row["word"]
    overall_df.at[i, "p(word|spam)"] = ((row["spam_count"] + 1) / (total_words_spam + m))
    overall_df.at[i, "p(word|ham)"] = ((row["ham_count"] + 1) / (total_words_ham + m))

In [13]:
overall_df.head()

,word,spam_count,ham_count,p(word|spam),p(word|ham)
0,forward,1258,467,0.001435,0.000373
1,support,148,365,0.00017,0.000292
2,kitchen,4,522,0.000006,0.000417
3,heard,32,346,0.000038,0.000276
4,december,209,469,0.000239,0.000374


### Top 5 Largest Values

In [14]:
spam_sorted = overall_df.sort_values(by='p(word|spam)', ascending=False)
ham_sorted = overall_df.sort_values(by='p(word|ham)', ascending=False)

spam_sorted = spam_sorted.drop(columns=['spam_count', 'ham_count', 'p(word|ham)'] )
ham_sorted = ham_sorted.drop(columns=['spam_count', 'ham_count', 'p(word|spam)'] )
# convert to dataframes
spam_sorted = pd.DataFrame(spam_sorted)
ham_sorted = pd.DataFrame(ham_sorted)

In [15]:
ham_sorted.head(5)


,word,p(word|ham)
513,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,0.052668
526,enron,0.040647
222,the,0.035941
578,to,0.026217
464,a,0.019219


In [16]:
spam_sorted.head(5)


,word,p(word|spam)
526,enron,0.036576
464,a,0.024309
222,the,0.022994
311,corp,0.020804
578,to,0.019564


# 4. Use these probabilities to classify

In [17]:
# Build word count dictionaries (used by the scorer) and keep the previously computed probability dicts for reference
ham_counts = overall_df.set_index('word')['ham_count'].to_dict()
spam_counts = overall_df.set_index('word')['spam_count'].to_dict()

# Keep the probability dicts (these were computed with add-1 Laplace earlier).
ham_prob_dict = ham_sorted.set_index('word')['p(word|ham)'].to_dict()
spam_prob_dict = spam_sorted.set_index('word')['p(word|spam)'].to_dict()

In [18]:

PRIOR_HAM = P_ham
PRIOR_SPAM = P_spam


In [19]:
import math

def calculate_log_scores(token_counts, coef_m, ham_counts, spam_counts, V):
    """
    Calculate log-probability scores for ham and spam using add-alpha smoothing
    where alpha = coef_m.

    P(word|class) = (count_in_class + alpha) / (total_words_in_class + alpha * V)

    Arguments:
    - token_counts: dict of word->count for the email
    - coef_m: smoothing parameter alpha
    - ham_counts, spam_counts: dicts with global counts per word in each class
    - V: vocabulary size (m)
    """

    # Safe log of priors (avoid log(0))
    log_score_ham = math.log(PRIOR_HAM) if PRIOR_HAM > 0 else float('-inf')
    log_score_spam = math.log(PRIOR_SPAM) if PRIOR_SPAM > 0 else float('-inf')

    denom_ham = (total_words_ham + coef_m * V)
    denom_spam = (total_words_spam + coef_m * V)

    for word, count in token_counts.items():
        # class counts (0 if unseen in that class)
        hc = ham_counts.get(word, 0)
        sc = spam_counts.get(word, 0)

        # add-alpha (generalized Laplace) smoothing for both seen and unseen words
        prob_ham = (hc + coef_m) / denom_ham
        prob_spam = (sc + coef_m) / denom_spam

        # accumulate log-probabilities (count times)
        log_score_ham += count * math.log(prob_ham)
        log_score_spam += count * math.log(prob_spam)

    return log_score_ham, log_score_spam

# The predict_class function remains unchanged
def predict_class(log_score_ham, log_score_spam):
    if log_score_ham > log_score_spam:
        return 'ham'
    else:
        return 'spam'

In [20]:
# Create a list to store prediction results

def predict_and_evaluate(df_test, coef_m):
    """
    Predict labels for df_test using smoothing alpha=coef_m.
    Returns a DataFrame with prediction results.
    """
    predictions_local = []

    for index, row in df_test.iterrows():
        token_counts = row['token_counts']
        true_label = row['label']

        # Calculate log scores using the parameterized scorer and the global class counts
        log_h, log_s = calculate_log_scores(token_counts, coef_m, ham_counts, spam_counts, m)

        # Predict class
        predicted_label = predict_class(log_h, log_s)

        # Store the result
        predictions_local.append({
            'email_id': row['email_id'],
            'true_label': true_label,
            'predicted_label': predicted_label,
            'log_ham_score': log_h,
            'log_spam_score': log_s,
            'is_correct': (true_label == predicted_label)
        })

    # Convert results to the final DataFrame
    df_results_local = pd.DataFrame(predictions_local)
    return df_results_local

# Example: run with coef_m=1
# df_results = predict_and_evaluate(df_test, 1)
# df_results.head()

In [21]:
# --- Calculate Accuracy ---
df_results_local = predict_and_evaluate(df_test, 1)
correct_predictions = df_results_local['is_correct'].sum()
total_emails = len(df_results_local)
accuracy = (correct_predictions / total_emails) * 100

print("\n------------------------------------")
print("📊 Final Model Evaluation Report")
print("------------------------------------")
print(f"Total test emails: {total_emails}")
print(f"Number of correct predictions: {correct_predictions}")
print(f"**Model Accuracy: {accuracy:.2f}%**")


------------------------------------
📊 Final Model Evaluation Report
------------------------------------
Total test emails: 1000
Number of correct predictions: 896
**Model Accuracy: 89.60%**


# 5. Vary the m parameter

In [22]:
for coef_m in [1, 10, 100, 10000]:
    df_results = predict_and_evaluate(df_test, coef_m)
    correct_predictions = df_results['is_correct'].sum()
    total_emails = len(df_results)
    accuracy = (correct_predictions / total_emails) * 100
    print(f"coef_m={coef_m}  Model Accuracy: {accuracy:.4f}%")

coef_m=1  Model Accuracy: 89.6000%
coef_m=10  Model Accuracy: 90.0000%
coef_m=100  Model Accuracy: 88.0000%
coef_m=10000  Model Accuracy: 79.1000%


## 🔧 Effect of the Smoothing Parameter *m* on Test Accuracy

### 1. Small *m* (Under-smoothing)
- Causes **overfitting**.  
- Probabilities for unseen words remain extremely low → model becomes overly sensitive to noise.  
- **Test accuracy drops** due to poor generalization.

---

### 2. Large *m* (Over-smoothing)
- Causes **underfitting**.  
- Excessive smoothing makes \( P(w|\text{Ham}) \approx P(w|\text{Spam}) \).  
- Classifier loses the ability to distinguish messages → relies mostly on class priors.  
- **Accuracy decreases significantly**.

---

### 3. Optimal *m*
- Achieved by methods like **cross-validation**.  
- Balances observed data with prior beliefs.  
- Yields the **highest generalization accuracy** on the test set.


# 6. If you were a spammer

## 1. 💧 Word Poisoning (Dilution)

**Goal:** Significantly increase the *Ham* score.  

**Method:**  
Inject a large volume of common, non-spam words (e.g., *“work”*, *“meeting”*, *“hello”*) into an otherwise spammy message.

**Effect:**  
Because many of these frequent words have a high probability \( P(w \mid \text{Ham}) \), the overall \( \text{Score}_{\text{Ham}} \) rises sharply, overpowering the negative contribution of the spam-related keywords.


---

## 2. 🎭 Obfuscation of Spam Keywords

**Goal:** Neutralize the impact of strong spam-indicator words.

**Method:**  
Alter common spam-triggering words (such as *“free”*, *“money”*) by applying misspellings, punctuation changes, or character substitutions (e.g., *"raygan"*, *"p0ol"*).

**Effect:**  
These modified words are treated as out-of-vocabulary (OOV) tokens by the classifier.  
Such tokens receive a neutral and usually low estimated probability (often via an \( m \)-estimate), which weakens their ability to push the message toward the spam class.


---

## 3. 🧠 Exploiting the Independence Assumption

**Goal:** Take advantage of the fundamental independence assumption of Naive Bayes.

**Method:**  
Embed strong spam keywords inside long, seemingly harmless sentences filled with neutral or ham-like words.

**Effect:**  
Because the classifier treats each word independently, the sum of the positive log-likelihood contributions from numerous neutral/ham words outweighs the negative, concentrated contribution of the spam keywords.  
As a result, the classifier is more likely to label the message as *Ham*.
